Последний принцип проектирование — dependency inversion principle. Принцип инверсии зависимостей.
Конкретные реализации должны зависить от абстракций, а не наоборот.

Пример нарушения (даже объекты не нужны):

In [6]:
def print_json(data):
    print('{"data":"' + data + '"}')
    
def print_xml(data, root):
    print('<{root}><data>{data}</data></{root}>'.format(
        root=root,
        data=data
    ))

def make_report(numbers, output, xml_root=None):
    data = 'Sum is {}'.format(sum(numbers))
    if output == 'json':
        print_json(data)
    elif output == 'xml':
        print_xml(data, xml_root)

In [7]:
make_report([1, 2, 3, 4], 'json')

{"data":"Sum is 10"}


In [8]:
make_report([1, 2, 3, 4], 'xml', 'some_root')

<some_root><data>Sum is 10</data></some_root>


Здесь у нас имеется:

* Одна абстракция ("выводильщик данных")
* Две её конкретных реализации (`print_json`, `print_xml`)
* Ещё одна абстракция ("выводильщик отчётов")
* Одна её конкретная реализация (`make_report`).

И получается, что абстрацкия "выводильщик отчётов" зависит от конкретных реализаций выводильщиков данных, потому что любой выводильщик отчётов обязан брать в качестве параметра `xml_root`, чтобы передать его конкретному `print_xml`.
Более того: конкретный `make_report` зависит от конкретного списка реализаций выводильшиков данных: он принимает параметр `output` и делает кучу if'ов, чтобы выбрать нужного выводильщика.
Чем это плохо:

1. Нельзя просто так взять и добавить ещё одного выводильщика. Особенно если он хочет дополнительные параметры.
2. Нельзя просто так взять и добавить параметров к `print_json`, потому что их потребуется добавлять в `make_report`.

Решение проблемы — сделать так, чтобы каждый конкретный выводильщик отчётов зависил лишь от абстракции "выводильщик данных". Для этого нужно её придумать. Например, давайте скажем, что "выводильщик данных" — это функция с одним параметром `data`:

In [9]:
def print_json(data):  # Сразу выводильщик данных
    print('{"data":"' + data + '"}')
    
def create_print_xml(root):  # Создаём выводильщика с конкретным root
    def print_xml(data):
        print('<{root}><data>{data}</data></{root}>'.format(
            root=root,
            data=data
        ))
    return print_xml

А после этого скажем, что конкретный `make_report` принимает любого абстрактного выводильщика данных:

In [10]:
def make_report(numbers, formatter):
    data = 'Sum is {}'.format(sum(numbers))
    formatter(data)

In [11]:
make_report([1, 2, 3, 4], print_json)

{"data":"Sum is 10"}


In [12]:
make_report([1, 2, 3, 4], create_print_xml('some_root'))

<some_root><data>Sum is 10</data></some_root>


Почему "инверсия зависимостей"? Потому что раньше у нас абстракция выводильщика отчётов зависела от конкретных выводильщиков данных, а теперь наоборот: конкретный выводильщик отчётов зависи от абстрактного выводильщика данных.

На бытовом уровне можно сформулировать так: если вы внутри функции/метода/класса напрямую обращаетесь к другому классу, это не очень хорошо, потому что вы зависите от конкретной реализации. Один из способов от этого избавиться — начать зависеть от абстракции, а конкретную реализацию получить как параметр конструктора. Это, кстати, называется [внедрение зависимостей](https://ru.wikipedia.org/wiki/%D0%92%D0%BD%D0%B5%D0%B4%D1%80%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D0%B8).